In [6]:
import os
import pickle
import numpy as np

from scopt import scopt
import phase_retrival as pr

# TODO
1. vectorize phase functions
2. add extra params to acelerate

## Data load

In [7]:
data_folder = '../../data'
problem_name = 'phase_data_128_32'
results = {problem_name: {}}

In [8]:
# load data
with open(os.path.join(data_folder, problem_name + '.pckl'), 'rb') as f:
    data = pickle.load(f)
    
a = data['a']
x_sol = data['x_sol']
x_0 = data['x_0']
A = np.array([np.dot(a_i.reshape(-1, 1), np.conj(a_i).reshape(1, -1)) for a_i in a])
X_sol = np.dot(x_sol.reshape(-1, 1), np.conj(x_sol).reshape(1, -1))
X_0 = np.dot(x_0.reshape(-1, 1), np.conj(x_0).reshape(1, -1))

lambda_val = [pr.norm(np.vdot(a_i, x_sol)) for a_i in a]
y = np.array([np.random.poisson(lambda_i) for lambda_i in lambda_val])

#y = data['y']
c = np.mean(y)

## Parameters initialization

In [9]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-10

FW_params={
    'iter_FW': 50000,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

# Auxilary functions

In [10]:
func_x = lambda X: pr.phase_val(A, X, y)
grad_x = lambda X, trace_sum: pr.phase_gradient(A, X, y, trace_sum)
#grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson(W, y, lam, (1 - beta) * x + beta * x, (1 - beta) * dot_product + beta * dot_product_s)
hess_mult_x = lambda S, X: pr.hess_mult(A, X, y, S)
hess_mult_vec_x = lambda S, X: pr.hess_mult_vec(A, X, y, S)
#extra_func = lambda x: W @ x
#linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
prox_func = lambda X, L: pr.proj_map(X, c) #used for SCOPT

## Run SCOPT

In [11]:
 X, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          X_0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[problem_name]['scopt'] = {
    'x': X,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

iter =    1, stepsize = 1.000e+00, rdiff = 0.000e+00 , f = -112326

Convergence achieved!
iter =    2, stepsize = 1.000e+00, rdiff = 0.000e+00,value=-112326

54.0025429725647
